In [26]:
#all the imports
import os, os.path
import qml
import math
from itertools import islice
import numpy as np
import random
import matplotlib.pyplot as plt
from qml.kernels import gaussian_kernel
from qml.kernels import laplacian_kernel
from qml.math import cho_solve
import seaborn as sns
from sklearn.utils import shuffle
from sklearn import metrics
from sklearn.kernel_ridge import KernelRidge
import statistics
from qml.kernels import matern_kernel
from sklearn.model_selection import train_test_split

In [27]:
x_benzene=np.load("Binaries/coulomb_matrix_list_tdm.npy")
y_benzene = np.loadtxt("InitialData/D_01.dat")


In [28]:
from sklearn.model_selection import train_test_split
from qml.math import cho_solve
def krr_qml(x_train,x_test,y_train,y_test,sigma):
    K=gaussian_kernel(x_train,x_train,sigma=sigma)
    K[np.diag_indices_from(K)] += 1e-9
    alpha = cho_solve(K, y_train)
    Ks=gaussian_kernel(x_train,x_test,sigma)
    y_predict=np.dot(alpha,Ks) 
    return np.mean(np.abs(y_predict - y_test))


In [52]:
# Split data into training and testing sets
X_train1, X_test1, y_train1, y_test1 = train_test_split(x_benzene, y_benzene[:,2], shuffle=False, train_size=0.8)

sigmas= range(1, 5000, 100)

def hypertuning(sigmas):
    """
    Hypertuning function which finds the optimal kernel width for a given dataset. The function minimises the MAE of KRR.

    Parameters:
        sigmas(range): The kernel widths to try. 
        
    Returns:
        optimal_sigma(int): The kernel width with the lowest MAE.
    """
    min_mae=np.inf
    optimal_sigma=None
    for sigma_val in sigmas:
        mae_templ = krr_qml(X_train1[:200],X_test1[:-50],y_train1[:200],y_test1[:-50],sigma_val)
        if mae_templ < min_mae:
            min_mae = mae_templ
            optimal_sigma = sigma_val
    return optimal_sigma    
   

print("Optimal Sigma value:",hypertuning(sigmas))  


Optimal Sigma value: 4901
<class 'int'>
